In [1]:
import shutil
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from tensorflow.keras.models import load_model, Model
from tasks.utils.tf_data_utils import build_data_pipeline

In [7]:
def load_saved_model(model_path: str):
    model = load_model(model_path)
    return model

In [8]:
model = load_saved_model('/home/ariya/workspace/models/animals10_classifier_second')

In [105]:
annotation_df = pd.read_csv('/home/ariya/workspace/datasets/animals10-dvc/annotation_df.csv')
classes = ["butterfly", "cat", "chicken", "cow", "dog", 
        "elephant", "horse", "sheep", "spider", "squirrel"]
img_size = [50,50]
batch_size = 1

test_ds = build_data_pipeline(annotation_df, classes, 'test', img_size, batch_size, 
                                do_augment=False, augmenter=None)
y_true_bin = np.concatenate([y for _, y in test_ds], axis=0)
y_true = np.argmax(y_true_bin, axis=1)

In [106]:
y_pred_prob = model.predict(test_ds)

2618/2618 [==============================] - 10s 4ms/step


In [107]:
y_pred = tf.argmax(y_pred_prob, axis=1)

In [108]:
y_true.shape, y_pred.shape

((2618,), TensorShape([2618]))

In [6]:
classes = ["butterfly", "cat", "chicken", "cow", "dog", 
        "elephant", "horse", "sheep", "spider", "squirrel"]
img_size = [50,50]
batch_size = 1

In [7]:
# mock up multi-label pred
y_pred_prob = np.random.rand(10,10)
y_pred = (y_pred_prob > 0.7).astype(np.int8)
y_true = (y_pred_prob > 0.5).astype(np.int8)

In [8]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, roc_auc_score

In [9]:
# Calculate precision, recall, f1-score
type = 'multi-label'
if type == 'multi-class':
    
    # Create a confusion matrix
    conf_matrix = confusion_matrix(y_true, y_pred)
    
    # Plot the confusion matrix
    fig = plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=True,
               xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted')
    plt.ylabel('Ground Truth')
    plt.title('Confusion Matrix')
    plt.show()
    
    # Calculate AUC
    roc_auc = roc_auc_score(y_true, y_pred_prob, average='macro', multi_class='ovr')
    
    # Print classification report
    report = classification_report(y_true, y_pred, target_names=classes, output_dict=True)
    final_report = pd.DataFrame(report).T
    print('AUC:', roc_auc)
    
elif type == 'multi-label':
    
    roc_auc = roc_auc_score(y_true, y_pred_prob, average=None, multi_class='ovr')
    
    # Print classification report
    report = classification_report(y_true, y_pred, target_names=classes, output_dict=True)
    final_report = pd.DataFrame(report).T
    final_report['AUC'] = list(roc_auc) + (4*[None])

final_report    

/opt/anaconda3/envs/computer-viz-dl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support,AUC
butterfly,1.000000,1.000000,1.000000,3.0,1.0
cat,0.000000,0.000000,0.000000,2.0,1.0
chicken,1.000000,0.857143,0.923077,7.0,1.0
cow,1.000000,0.400000,0.571429,5.0,1.0
dog,1.000000,0.600000,0.750000,5.0,1.0
elephant,1.000000,0.571429,0.727273,7.0,1.0
horse,1.000000,0.600000,0.750000,5.0,1.0
sheep,1.000000,0.600000,0.750000,5.0,1.0
spider,1.000000,1.000000,1.000000,2.0,1.0
squirrel,1.000000,0.666667,0.800000,6.0,1.0


In [13]:
final_report.reset_index()

,index,precision,recall,f1-score,support,AUC
0,butterfly,1.000000,1.000000,1.000000,3.0,1.0
1,cat,0.000000,0.000000,0.000000,2.0,1.0
2,chicken,1.000000,0.857143,0.923077,7.0,1.0
3,cow,1.000000,0.400000,0.571429,5.0,1.0
4,dog,1.000000,0.600000,0.750000,5.0,1.0
5,elephant,1.000000,0.571429,0.727273,7.0,1.0
6,horse,1.000000,0.600000,0.750000,5.0,1.0
7,sheep,1.000000,0.600000,0.750000,5.0,1.0
8,spider,1.000000,1.000000,1.000000,2.0,1.0
9,squirrel,1.000000,0.666667,0.800000,6.0,1.0


In [114]:
isinstance(roc_auc, float)

True

In [1]:
from importlib import import_module

In [2]:
mod = import_module('flows.full_flow', package='flows')

In [3]:
mod.start({'add_text': 'eiei'})

Hi Mom!
added eiei


---

In [71]:
import os
import cv2
import base64
import numpy as np
import requests
import random
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image

In [72]:
img_root = '/home/ariya/workspace/datasets/animals10-dvc/images'
img_class = 'butterfly'
img_dir = os.path.join(img_root, img_class)
img_file = random.choice(os.listdir(img_dir))
print(img_file)
img_path = os.path.join(img_dir, img_file)

ea35b80a2ef1043ed1584d05fb1d4e9fe777ead218ac104497f5c97faeebb5bb_640.jpg


In [83]:
ori_image = cv2.imread(img_path)

In [73]:
res = requests.post('http://127.0.0.1:4242/predict/',
                    files={'item':open(img_path, 'rb')}
                    )
print(res.json())

{'message': 'No model. You have to setup model with the /update_model endpoint first.'}


In [74]:
res = requests.put('http://127.0.0.1:4242/update_model/animals10_classifierXD.yaml')
print(res.json())

{'message': 'Updating model failed due to failure in model loading method'}


In [75]:
res = requests.put('http://127.0.0.1:4242/update_model/animals10_classifier.yaml')
print(res.json())

{'message': 'Update the model successfully'}


In [78]:
res = requests.post('http://127.0.0.1:4242/predict/',
                    files={'item':open(img_path, 'rb')}
                    )
res = res.json()
res.keys()

dict_keys(['prediction', 'overlaid_img', 'raw_hm_img', 'message'])

In [79]:
raw_hm_bytes = res.pop('raw_hm_img')
overlaid_hm_bytes = res.pop('overlaid_img')
res

{'prediction': {'butterfly': 0.969391942024231,
  'cat': 0.00346136512234807,
  'chicken': 5.091017374070361e-05,
  'cow': 3.8311532080115285e-06,
  'dog': 0.0029960433021187782,
  'elephant': 9.680839951897724e-08,
  'horse': 6.2787119532004e-05,
  'sheep': 3.5267048588139005e-06,
  'spider': 0.023964786902070045,
  'squirrel': 6.465009937528521e-05},
 'message': 'Success'}